<h1 style="font-family: Verdana; font-size: 28px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; background-color: #8DE7E3; color: black;"><center><br>MayoClinic: WSI Preprocessing + Tiling</center></h1>
                                                      
<center><img src = "https://drive.google.com/uc?id=1Xqf0NCEMGOFbfEZNtTXQHmMp63g3Qvko" width = "1000" height = "400"/></center>   

<h5 style="text-align: center; font-family: Verdana; font-size: 12px; font-style: normal; font-weight: bold; text-decoration: None; text-transform: none; letter-spacing: 1px; color: black; background-color: #ffffff;">CREATED BY: NGHI HUYNH</h5>

<p id="toc"></p>
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; background-color: #8DE7E3; color: black;" role="tab" aria-controls="home"><center><br>CONTENTS</center></h2>

<h3 style="text-indent: 10vw; font-family: Verdana; font-size: 16px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: black; background-color: #ffffff;"><a href="#motive">0&nbsp;&nbsp;&nbsp;&nbsp;MOTIVATION</a></h3>

---

<h3 style="text-indent: 10vw; font-family: Verdana; font-size: 16px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: black; background-color: #ffffff;"><a href="#wsi">1&nbsp;&nbsp;&nbsp;&nbsp;WSI ANALYSIS</a></h3>

---

<h3 style="text-indent: 10vw; font-family: Verdana; font-size: 16px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: black; background-color: #ffffff;"><a href="#preprocessing">2&nbsp;&nbsp;&nbsp;&nbsp;DATA PREPROCESSING</a></h3>

---

<h3 style="text-indent: 10vw; font-family: Verdana; font-size: 16px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: black; background-color: #ffffff;"><a href="#extract">3&nbsp;&nbsp;&nbsp;&nbsp;TILE SELECTION</a></h3>

---

<h3 style="text-indent: 10vw; font-family: Verdana; font-size: 16px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: black; background-color: #ffffff;"><a href="#conclusion">4&nbsp;&nbsp;&nbsp;&nbsp;CONCLUSION</a></h3>

---

<div class="list-group" id="list-tab" role="tablist">
<h3 class="list-group-item list-group-item-action active" data-toggle="list" style='background:#F08080; border:0; color:black' role="tab" aria-controls="home"><center><br>If you find this notebook useful, do give me an upvote, it motivates me a lot.<br><br> This notebook is still a work in progress. Keep checking for further developments!😊</center></h3>

<a id="motive"></a>

<h2 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; background-color: #8DE7E3; color: black;" id="motive"><left><br>&nbsp;0. MOTIVATION <a href="#toc">&#10514;</a><br></left> </h2>

# *Problems*
* **Whole Slide Images (WSIs)**: large (multiple GB in size) -> need to break up into tiles/patches to further analysis. (preprocessing)

![](https://drive.google.com/uc?id=1TGQZHZ9tBLSl2AaETQOXGI3r7BEWLYs-)

* **Artefacts due to slide preparation, large portions of background**: need to be removed to speed up the training process and potentially improve performance-> deep tissue detector (artifacts, background, tissue)

![](https://drive.google.com/uc?id=10L_Z375GmH4NW2jLz6fR9wVoZFpvd8RS)

# *Data preprocessing*
**1. Image Scaling:** scale down whole-slide images (WSIs) in the training data to 1024x1024 resolution

**2. Tissue Segmentation:** identify tissue regions in scaled-down images using the following foreground/background filtering approaches:
* [Otsu's Binarization](https://docs.opencv.org/4.x/d7/d4d/tutorial_py_thresholding.html)
* [Triangle Binarization](https://subscription.packtpub.com/book/data/9781789344912/9/ch09lvl1sec80/the-triangle-binarization-algorithm)
    
**3. Fast Tiling + Tile Selection:** select tiles based on thresholds calculated from step 2. Note that tile size should be large enough that feature relevant to the task are visible to the model to learn

**4. Tile Retrieval + Saving:** retrive and save selected tiles for further analysis (future work)

[**Reference:** Apply filters for tissue segmentation](https://developer.ibm.com/articles/an-automatic-method-to-identify-tissues-from-big-whole-slide-images-pt2/)


<a id="wsi"></a>

<h2 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; background-color: #8DE7E3; color: black;" id="wsi"><left><br>&nbsp;1. WSI ANALYSIS <a href="#toc">&#10514;</a><br></left> </h2>

# *Imports*

[OpenSlide](https://openslide.org/) can read virtual slides in the following formats:

* **Aperio (.svs, .tif)**: our image data is in the .tif format
* Philips (.tiff)
* Sakura (.svslide)
* Generic tiled TIFF (.tif)

In [ ]:
# the OpenSlide package needs C library dependencies
!apt-get install openslide-tools
# install openslide-python package in google colab
!pip install openslide-python

In [ ]:
import os
import sys
import logging
import time
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import tifffile as tiff
import pandas as pd
import gc
import math
import cv2
import seaborn as sns

import openslide # see installation guide which requires a 3rd party library
from openslide import open_slide
from openslide.deepzoom import DeepZoomGenerator

# *Helper functions*

In [ ]:
def rescale_wsi(slide, scale_factor):
    dims = slide.level_dimensions[0] #level 0 is the highest resolution
    large_w, large_h = slide.dimensions
    new_w = math.floor(large_w/scale_factor)
    new_h = math.floor(large_h/scale_factor)
    wsi = slide.read_region((0,0), 0, dims) #read region at level 0
    wsi = wsi.convert("RGB") #since img is in RGBA, need to convert to RGB for further preprocessing
    rescale_img = wsi.resize((new_w, new_h), Image.Resampling.BILINEAR) # PIL image
    return rescale_img

def create_property_df(slide):
    slide_properties = slide.properties
    descriptions = ['The number of levels in the slide. Levels are numbered from 0 (highest resolution) to level_count - 1 (lowest resolution)',
                    'A list of downsample factors for each level of the slide. level_downsamples[k] is the downsample factor of level k',
                    'Height at level k',
                    'Tile height at level k',
                    'Tile width at level k',
                    'Width at level k',
                    'The name of the property containing an identification of the vendor',
                    'Resolution Unit of the slide',
                    'X Resolution',
                    'Y Resolution',
                    'A (width, height) tuple for level 0 of the slide',
                    'Image mode/attribute'
                   ]
    properties = {}
    smaller_region = slide.read_region((slide.dimensions[0]//2,slide.dimensions[1]//2), 0, (1024,1024))
    for key, value in slide_properties.items():
        properties[key] = value
    properties['slide-dimension'] = slide.dimensions # widthxheight
    properties['image-mode'] = smaller_region.mode
    df_properties = pd.DataFrame.from_dict(properties, orient='index').reset_index() #orient index is to create the df using dict keys as rows
    df_properties.columns = ['Slide Property', 'Value']
    df_properties['Description'] = descriptions
    df_properties = df_properties[['Slide Property','Description','Value']]
    return df_properties

def visualize_slide(df, loc, img):
    plt.figure(figsize=(30,10))
    plt.imshow(img)
    plt.title(f'Label: {df.loc[loc].label}\nImage ID: {df.loc[loc].image_id}\nCenter ID: {df.loc[loc].center_id}\nImage shape: {img.size}')
    plt.show()

def highlight(row):
    df = lambda x: ['background: #8DE7E3' if x.name in row
                        else '' for i in x]
    return df

# *Slide properties*

In [ ]:
#Load the slide file (svs, tif) into an object
slide = open_slide("../input/mayo-clinic-strip-ai/train/00c058_0.tif")
rescale_img = rescale_wsi(slide, 32)
df = create_property_df(slide)
df.style.hide_index().apply(highlight([0,2,5,7,11]), axis=1)

# *Slide visualization*

In [ ]:
df = pd.read_csv('../input/mayo-clinic-strip-ai/train.csv')
df.head().style.hide_index().apply(highlight([2]), axis=1)

In [ ]:
visualize_slide(df, 2, rescale_img)

<a id="preprocessing"></a>

<h2 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; background-color: #8DE7E3; color: black;" id="preprocessing"><left><br>&nbsp;2. DATA PREPROCESSING <a href="#toc">&#10514;</a><br></left> </h2>

# *Tissue Segmentation*

## Goal:¶
Mask out non-tissue by setting non-tissue pixels to 0 for their red, green, and blue channels.

Conceptually and mathematically, it is often useful to have background values close to or equal to 0 (complement image)

Complement image: simply subtract each pixel value from the maximum pixel value supported by the class (class uint8, the max value of pixel can be 255) and store in the output image array. In the output image, dark areas become lighter and light areas become darker

## Foreground/Background Thresholding Techniques:
* [Otsu](https://docs.opencv.org/4.x/d7/d4d/tutorial_py_thresholding.html): global image thresholding algorithm, applied to images which are bimodal, which means the image is having 2 peaks in the histogram
* [Triangle binarization](https://subscription.packtpub.com/book/data/9781789344912/9/ch09lvl1sec80/the-triangle-binarization-algorithm): a line is drawn from the highest bar to the end of the histogram. Then for choosing the optimal threshold, distance of each bar is calculated from the line, whichever is least becomes the threshold value

**Side Note:** [Understanding Histograms in Image Processing](https://towardsdatascience.com/histograms-in-image-processing-with-skimage-python-be5938962935)

# *Helper functions*

In [ ]:
# refer to my notebook: https://www.kaggle.com/code/nghihuynh/wsi-preprocessing-tiling-tissue-segmentation/notebook
def thresholding(img, method='otsu'):
    # convert to grayscale complement image
    grayscale_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_c = 255 - grayscale_img
    thres, thres_img = 0, img_c.copy()
    if method == 'otsu':
        thres, thres_img = cv2.threshold(img_c, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    elif method == 'triangle':
        thres, thres_img = cv2.threshold(img_c, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_TRIANGLE)
    return thres, thres_img, img_c

def histogram(img, thres_img, img_c, thres):
    """
    style: ['color', 'grayscale']
    """
    plt.figure(figsize=(15,15))

    plt.subplot(3,2,1)
    plt.imshow(img)
    plt.title('Scaled-down image')

    plt.subplot(3,2,2)
    sns.histplot(img.ravel(), bins=np.arange(0,256), color='orange', alpha=0.5)
    sns.histplot(img[:,:,0].ravel(), bins=np.arange(0,256), color='red', alpha=0.5)
    sns.histplot(img[:,:,1].ravel(), bins=np.arange(0,256), color='Green', alpha=0.5)
    sns.histplot(img[:,:,2].ravel(), bins=np.arange(0,256), color='Blue', alpha=0.5)
    plt.legend(['Total', 'Red_Channel', 'Green_Channel', 'Blue_Channel'])
    plt.ylim(0,0.05e6)
    plt.xlabel('Intensity value')
    plt.title('Color histogram')

    plt.subplot(3,2,3)
    plt.imshow(img_c, cmap='gist_gray')
    plt.title('Complement grayscale image')

    plt.subplot(3,2,4)
    sns.histplot(img_c.ravel(), bins=np.arange(0,256))
    plt.axvline(thres, c='red', linestyle="--")
    plt.ylim(0,0.05e6)
    plt.xlabel('Intensity value')
    plt.title('Grayscale complement histogram')

    plt.subplot(3,2,5)
    plt.imshow(thres_img, cmap='gist_gray')
    plt.title('Thresholded image')

    plt.subplot(3,2,6)
    sns.histplot(thres_img.ravel(), bins=np.arange(0,256))
    plt.axvline(thres, c='red', linestyle="--")
    plt.ylim(0,0.05e6)
    plt.xlabel('Intensity value')
    plt.title('Thresholded histogram')

    plt.tight_layout()
    plt.show()

# *Otsu's binarization*

In [ ]:
img_np = np.array(rescale_wsi(slide, 16))

In [ ]:
thres_otsu, thres_img, img_c = thresholding(img_np, method='otsu')
histogram(img_np, thres_img, img_c, thres_otsu)

# *Triangle binarization*

In [ ]:
thres_triangle, thres_img, img_c = thresholding(img_np, method='triangle')
histogram(img_np, thres_img, img_c, thres_triangle)

<a id="extract"></a>

<h2 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; background-color: #8DE7E3; color: black;" id="extract"><left><br>&nbsp;3. TILE EXTRACTION <a href="#toc">&#10514;</a><br></left> </h2>

# *Helper functions*

In [ ]:
def tile_properties(tiles):
    tile_properties = {'Level count': ['The number of Deep Zoom levels in the image',tiles.level_count],
            'Tile count': ['The total number of Deep Zoom tiles in the image',tiles.tile_count],
            'Level tiles': ['A list of (tiles_x, tiles_y) tuples for each Deep Zoom level. level_tiles[k] are the tile counts of level k',tiles.level_tiles],
            'Level dimensions': ['A list of (pixels_x, pixels_y) tuples for each Deep Zoom level. level_dimensions[k] are the dimensions of level k',tiles.level_dimensions]
           }
    tile_df = pd.DataFrame.from_dict(tile_properties, orient='index').reset_index()
    tile_df.columns = ['Tile property', 'Description', 'Value']
    return tile_df

def visualize_tiles(tiles, level, thres, tile_size):
    rows, cols = tiles.level_tiles[level]
    plt.figure(figsize=(10,10))
    count = 0
    img_size = 0
    for row in range(rows):
        for col in range(cols):
            count += 1
            plt.subplot(4,4,count)
            plt.xticks([])
            plt.yticks([])
            single_tile = tiles.get_tile(level, (row,col))
            single_tile_RGB = single_tile.convert('RGB')
            img_test = np.array(single_tile_RGB.resize((tile_size, tile_size), Image.Resampling.BILINEAR))
            img_size = img_test.shape
            plt.imshow(img_test)
            plt.tight_layout()
    plt.suptitle(f'Tile level: {level}\nNum tiles: {count}\nTile size: {img_size}')
    plt.tight_layout()
    plt.show()

    plt.figure(figsize=(10,10))
    count = 0
    selected_tiles = 0
    img_size = 0
    for row in range(rows):
        for col in range(cols):
            count += 1
            plt.subplot(4,4,count)
            plt.xticks([])
            plt.yticks([])
            single_tile = tiles.get_tile(level, (row,col))
            single_tile_RGB = single_tile.convert('RGB')
            img_test = np.array(single_tile_RGB.resize((tile_size, tile_size), Image.Resampling.BILINEAR))
            img_size = img_test.shape
            img_c = 255 - img_test
            if img_c.mean() > thres:
                selected_tiles += 1
                plt.imshow(img_test)
                plt.tight_layout()
    plt.suptitle(f'Tile level: {level}\nNum selected tiles: {selected_tiles}\nTile size: {img_size}')
    plt.tight_layout()
    plt.show()


# *Tile properties*

In [ ]:
tiles = DeepZoomGenerator(slide, tile_size=1024, overlap=0, limit_bounds=False)
tile_df = tile_properties(tiles)
tile_df.style.hide_index().apply(highlight([1,3]), axis=1)

> #### *Working with lower level dimensions might be beneficial for quick prototyping. In contrast, we should be cautious that higher level dimensions yield more tiles with a given tile size. If we worry about OOR, we should start with lower dimensions and build up from that.*

# *Tile selection*

In [ ]:
visualize_tiles(tiles, 13, thres_triangle, 1024)

> #### *We can discard many uneccessary tiles such as background or artefacts after these thresholding techniques. However, there's still a large portion of background in the selected tiles. We might need to choose smaller tile sizes to increase the quality of the training data.*
> #### *WSI are often labeled at the slide level, not at the tile level. Thus, we might need to annotate them at tile level to derive the ground truth labels of individual tiles from slide-wide label data*

<a id="conclusion"></a>

<h2 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; background-color: #8DE7E3; color: black;" id="conclusion"><left><br>&nbsp;4. CONCLUSION <a href="#toc">&#10514;</a><br></left> </h2>

## Recap: WSI preprocessing framework:
### Steps:
1. **Scale** down images

2. **Filter** foreground/background (tissue/non-tissue)

3. **Break** down images into **tiles**, and **select** tiles containing large portion of **tissue regions**

4. **Retrieve** and **save** selected tiles for further analysis (future work)

### Remarks:

* Working with **lower level dimensions** might be beneficial for quick prototyping. In contrast, we should be cautious that **higher level dimensions** yield more tiles with a given tile size. If we worry about OOR, we should start with lower dimensions and build up from that.
* We can discard many **uneccessary tiles** such as background or artefacts after these thresholding techniques. However, there's still a large portion of background in the selected tiles. We might need to choose smaller **tile sizes** to increase the quality of the training data.
* WSI are often labeled at the **slide level**, not at the tile level. Thus, we might need to **annotate** them at **tile level** to derive the ground truth labels of individual tiles from slide-wide label data